# ECM calculation in several molecules:
Normal vectors and achiral molecules can be taken from https://csm.ouproj.org.il/molecule


**Task:** 
1. Define the molecules that will be under study as well as their associated defining vectors
2. Define the vectors which are supposed to be perpendicular to the mirroring plane (random vectors are taken here)





In [1]:
# creating the path (PYTHONPATH) to our module.
# assuming that our 'pyECM' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [23]:
import numpy as np
from pyECM.molecule_class import molecula



N_molecules = 2
names_molecules = [None]*N_molecules
vectores = [None]*N_molecules


vectores[0]= np.array([-0.0172, 0.0068, 0.9998])
names_molecules[0] = 'CFMAR'

vectores[1]= np.array([-0.0083, 0.0045, 1.0000])
names_molecules[1] = 'AP1'

#Define mirror path interval
minimo = 0.75
maximo = 1.0
delta = 0.05

puntos = int (round( (maximo - minimo)/delta)) + 1
#Generates the files associated with the mirror path
for i in range(N_molecules):
    mol_achiral = molecula(XYZ_file = '../pyECM/data/import/'+names_molecules[i]+'_achiral.xyz')
    random_atom_achiral = mol_achiral.positions[0]
    mol_chiral = molecula(XYZ_file = '../pyECM/data/import/'+names_molecules[i]+'_chiral.xyz', dipolo=vectores[i], origen=random_atom_achiral)
    mol_chiral.rotate_to_align_dipole_with_z()
    mol_chiral.xyz_mirror_path(folder='../pyECM/data/export/', prefix_name=names_molecules[i], lim_inf=minimo, lim_sup=maximo, points=puntos, DIRAC = True)



In [18]:
import pandas as pd

#Calculate and save ECM & CCM

minimo = 0.75
maximo = 1.0
delta = 0.05
puntos = int (round( (maximo - minimo)/delta)) + 1
#base='dyallcv2z'
base='sto-3g'

# El segundo índice está demás? (CHEQUEAR)
zrate = np.zeros([N_molecules,puntos])
Norms1 = np.zeros([N_molecules,puntos])
CCMs1 = np.zeros([N_molecules,puntos])
Norms2 = np.zeros([N_molecules,puntos])
CCMs2 = np.zeros([N_molecules,puntos])
ECMs_NR = np.zeros([N_molecules,puntos])
ECMs_4c = np.zeros([N_molecules,puntos])

dataframes = []

# Get and save CCM and ECM values (takes about 2 min)
for i in range(N_molecules):
    mol = molecula(XYZ_file='../pyECM/data/export/'+names_molecules[i]+'_1.00.xyz')
    zrate[i], Norms1[i], CCMs1[i], Norms2[i], CCMs2[i] = mol.CCM_on_path(lim_inf=minimo, lim_sup=maximo,points=puntos)
    zrate[i], ECMs_NR[i], ECMs_molcontr, ECMs_4c[i] = mol.ECM_on_path(name='../pyECM/data/export/'+names_molecules[i], lim_inf=minimo, lim_sup=maximo, points=puntos, tracking=False, basis_set = base, fourcomp=True, debug=0)
    

    matrix_aux = np.vstack([zrate[i],Norms1[i], CCMs1[i], Norms2[i], CCMs2[i], ECMs_NR[i], ECMs_4c[i]])
    matrix     = np.transpose(matrix_aux)

    matrix_aux2 = np.vstack([zrate[i], ECMs_molcontr])
    matrix_aux2 = np.transpose(matrix_aux2)


    #np.save('../results/'+names_molecules[i]+'_molcontr',matrix_aux2)
    np.save('../pyECM/data/results/'+names_molecules[i],matrix)


In [50]:
#Create Tables
import pandas as pd


#N_molecules = 1
dataframes = [] #List with dataframes
dataframes_original_mols = []
dataframes_perorbital = []

for i in range(N_molecules):
    #Import CCM & ECM obtained results 
    dataframes.append(pd.DataFrame(np.load('../pyECM/data/results/'+names_molecules[i]+'.npy')))
    dataframes[i].columns = ['z rate', 'Norms 1', 'CCM 1', 'Norms 2', 'CCM 2', 'ECM_NR', 'ECM_4c']

    #Import HPV & chir dens obtained results
    HPV = pd.read_csv('results/H_PV/Tablas/TotalValues_'+names_molecules[i]+'.tab', sep="\t", header=None, skiprows=2)[2]
    chir_dens = pd.read_csv('results/H_PV/Tablas/TotalValues_'+names_molecules[i]+'.tab', sep="\t", header=None, skiprows=2)[3]
    zrate_check=pd.read_csv('results/H_PV/Tablas/TotalValues_'+names_molecules[i]+'.tab', sep="\t", header=None, skiprows=2)[1]
    if ((dataframes[i]['z rate']-zrate_check).all() != 0):
        raise Exception("Z rate of dataframes are not the same")
    dataframes[i]['HPV'] = HPV
    dataframes[i]['|HPV|'] = np.abs(HPV)
    dataframes[i]['Chir Dens'] = chir_dens

    #Import chirdens & ECM per orbital results
    dataframes_perorbital.append(pd.DataFrame(np.load('results/CCM&ECM/'+names_molecules[0]+'_molcontr.npy')))

    #Dataframe for original chiral ligands
    contain_values = pd.DataFrame(dataframes[i][dataframes[i]['z rate'].astype(str).str.contains('1.0')])
    contain_values['Ligand'] = names_molecules[i]
    contain_values['|HPV|'] = np.abs(HPV)
    contain_values['|Chir Dens|'] = np.abs(chir_dens)
    dataframes_original_mols.append(contain_values)

#Export results to xlsx file
with pd.ExcelWriter('MirrorPaths.xlsx') as writer:
    for indice in range(N_molecules):
        dataframes[indice].to_excel(writer,sheet_name=names_molecules[indice], index=False)

all_mol = pd.concat(dataframes_original_mols)
with pd.ExcelWriter('output_toghether.xlsx') as writer:
    all_mol.to_excel(writer, index=False)


with pd.ExcelWriter('Analysis_per_orbital.xlsx') as writer:
    for indice in range(N_molecules):
        dataframes_perorbital[indice].to_excel(writer,sheet_name=names_molecules[indice], index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'results/H_PV/Tablas/TotalValues_A.tab'